<a href="https://colab.research.google.com/github/anna-alt/AI-Lab/blob/main/SceneRecognition3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, SubsetRandomSampler, ConcatDataset
from torchvision import datasets, transforms
from torchvision import models
import os
import numpy as np
import sklearn
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

device

device(type='cpu')

In [21]:
trainset_size = 1000
val_size = 100
num_epochs = 10
image_size = (256,256)


In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Resize(image_size),
                              
                              ])
# Download and load the training data
trainset = datasets.SUN397('/content/drive/MyDrive/Data', download= False, transform=transform)
splittrain = torch.utils.data.random_split(trainset, [trainset_size, len(trainset) - trainset_size])[0]
train_loader = DataLoader(splittrain, batch_size=64,num_workers = 2, shuffle=True)

validationset = datasets.SUN397("/content/drive/MyDrive/Data", download=False, transform=transform)
splitvalidation = torch.utils.data.random_split(validationset, [val_size, len(trainset) - val_size])[0]
val_loader = DataLoader(splitvalidation, batch_size=64,num_workers = 2, shuffle=True)



In [56]:
classes = list()


with open("/content/drive/MyDrive/Data/SUN397/ClassName.txt") as class_file:
      for line in class_file:
          line = line.split()[0]
          split_indices = [i for i, letter in enumerate(line) if letter == '/']
          # Check if there a class with a subclass inside (outdoor, indoor)
          if len(split_indices) > 2:
              line = line[:split_indices[2]] + '-' + line[split_indices[2]+1:]

              classes.append(line[split_indices[1] + 1:])
          
        # Get number of classes
print(len(classes))

print(classes)

filenames = list()
labels = list()
labelsindex = list()
with open("/content/drive/MyDrive/Data/SUN397/ClassName.txt") as class_file:
      for line in class_file:
          line = line.split()[0]
          split_indices = [i for i, letter in enumerate(line) if letter == '/']
          # Check if there a class with a subclass inside (outdoor, indoor)
          
              
             
          name = line[split_indices[1] + 1:-1]
          label = line[split_indices[0] + 1: split_indices[1]]

          filenames.append(name)
          labels.append(label)
                          
          

print(labels)

126
['apartment_building-outdoor', 'apse-indoor', 'arrival_gate-outdoor', 'athletic_field-outdoor', 'atrium-public', 'badminton_court-indoor', 'bakery-shop', 'balcony-exterior', 'balcony-interior', 'basketball_court-outdoor', 'bazaar-indoor', 'bazaar-outdoor', 'bistro-indoor', 'booth-indoor', 'bow_window-indoor', 'bow_window-outdoor', 'brewery-indoor', 'cabin-outdoor', 'canal-natural', 'canal-urban', 'car_interior-backseat', 'car_interior-frontseat', 'casino-indoor', 'cathedral-indoor', 'cathedral-outdoor', 'cavern-indoor', 'chicken_coop-indoor', 'chicken_coop-outdoor', 'church-indoor', 'church-outdoor', 'cloister-indoor', 'control_tower-outdoor', 'covered_bridge-exterior', 'cubicle-office', 'desert-sand', 'desert-vegetation', 'diner-indoor', 'diner-outdoor', 'dinette-home', 'dinette-vehicle', 'doorway-outdoor', 'driving_range-outdoor', 'elevator-door', 'elevator-interior', 'escalator-indoor', 'factory-indoor', 'field-cultivated', 'field-wild', 'firing_range-indoor', 'florist_shop-indo

In [ ]:
model = models.resnet50(weights = "DEFAULT")
model.to(device)

In [18]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.999), eps=1e-8)
criterion = nn.CrossEntropyLoss()


In [25]:
def train(epoch, log_interval=20):
    # Set model to training mode
    model.train()
    
    # Loop over each batch from the training set
    for batch_idx, (data, target) in enumerate(train_loader):
        # Copy data to GPU if needed
        data = data.to(device)
        target = target.to(device)

        # Zero gradient buffers
        optimizer.zero_grad() 
        
        # Pass data through the network
        output = model(data)

        # Calculate loss
        loss = criterion(output, target)

        # Backpropagate
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))

In [27]:
def view_classify(img, ps):
    ''' Function for viewing an image and it's predicted classes.
    '''
    ps = ps.data.numpy().squeeze()

    fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
    ax1.imshow(img.resize_(1, 28, 28).numpy().squeeze())
    ax1.axis('off')
    ax2.barh(np.arange(10), ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))
    
    ax2.set_yticklabels(np.arange(10))

    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()

In [67]:
def show_sample(img, label):
    print("Label:", classes[label])
    plt.imshow(img.permute(1, 2, 0))

In [69]:
dataiter = iter(train_loader)
images,label = next(dataiter)
img_idx = 0

show_sample(trainset.__getitem__(50),50)

TypeError: ignored

In [26]:
from sklearn import metrics
def validate(loss_vector, accuracy_vector):
    model.eval()
    val_loss, correct = 0, 0
    for data, target in val_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        val_loss += criterion(output, target).data.item()
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()
        print(metrics.classification_report(target.cpu(), pred.cpu(),zero_division=0))
    val_loss /= len(val_loader)
    loss_vector.append(val_loss)

    

    accuracy = 100. * correct.to(torch.float32) / len(val_loader.dataset)
    accuracy_vector.append(accuracy)

        
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(val_loader.dataset), accuracy))


In [ ]:
#Definisco una funzione per stampare la label predetta e quella corretta

def Predic(loss_vector,accuracy_vector):
    val_loss, correct = 0, 0
    for data, target in val_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        val_loss += criterion(output, target).data.item()
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()
    print( pred)
    print("/n")
    print(target)


In [ ]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [ ]:
lossv, accv = [], []
for epoch in range(1, num_epochs + 1):
    train(epoch)
    validate(lossv, accv)
    Predic(lossv,accv)

Train Epoch: 1 [0/25000 (0%)]	Loss: 2.055968
Train Epoch: 1 [1280/25000 (5%)]	Loss: 1.785924
Train Epoch: 1 [2560/25000 (10%)]	Loss: 1.649735
Train Epoch: 1 [3840/25000 (15%)]	Loss: 1.793809
Train Epoch: 1 [5120/25000 (20%)]	Loss: 1.393523


/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1385478 bytes but only got 23143. Skipping tag 513
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 7276331008 bytes but only got 0. Skipping tag 7
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1879769088 bytes but only got 0. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3014066176 bytes but only got 0. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3568893952 bytes but only got 23144. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-pac

Train Epoch: 1 [6400/25000 (26%)]	Loss: 1.821659
Train Epoch: 1 [7680/25000 (31%)]	Loss: 1.153352
Train Epoch: 1 [8960/25000 (36%)]	Loss: 1.564478
Train Epoch: 1 [10240/25000 (41%)]	Loss: 1.886395
Train Epoch: 1 [11520/25000 (46%)]	Loss: 1.405707
Train Epoch: 1 [12800/25000 (51%)]	Loss: 1.721627
Train Epoch: 1 [14080/25000 (56%)]	Loss: 1.751145
Train Epoch: 1 [15360/25000 (61%)]	Loss: 1.591672
Train Epoch: 1 [16640/25000 (66%)]	Loss: 1.515297
Train Epoch: 1 [17920/25000 (72%)]	Loss: 1.894361
Train Epoch: 1 [19200/25000 (77%)]	Loss: 1.370256
Train Epoch: 1 [20480/25000 (82%)]	Loss: 1.184388
Train Epoch: 1 [21760/25000 (87%)]	Loss: 1.454475
Train Epoch: 1 [23040/25000 (92%)]	Loss: 1.139738
Train Epoch: 1 [24320/25000 (97%)]	Loss: 1.463569
              precision    recall  f1-score   support

           6       0.50      1.00      0.67         1
           7       1.00      1.00      1.00         1
           8       1.00      1.00      1.00         1
          24       0.00      0.00   

/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1385478 bytes but only got 23143. Skipping tag 513
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 7276331008 bytes but only got 0. Skipping tag 7
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1879769088 bytes but only got 0. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3014066176 bytes but only got 0. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3568893952 bytes but only got 23144. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-pac

Train Epoch: 2 [23040/25000 (92%)]	Loss: 0.596258
Train Epoch: 2 [24320/25000 (97%)]	Loss: 0.551093
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           2       1.00      1.00      1.00         1
          15       0.00      0.00      0.00         1
          22       0.00      0.00      0.00         1
          41       0.00      0.00      0.00         1
          48       1.00      1.00      1.00         1
          55       1.00      1.00      1.00         1
          56       0.50      0.50      0.50         2
          58       1.00      1.00      1.00         1
          60       1.00      1.00      1.00         1
          61       1.00      1.00      1.00         1
          64       1.00      1.00      1.00         1
          65       1.00      1.00      1.00         1
          78       1.00      1.00      1.00         1
          82       1.00      1.00      1.00         1
          83       1.00      1.00  

/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1385478 bytes but only got 23143. Skipping tag 513
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 7276331008 bytes but only got 0. Skipping tag 7
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1879769088 bytes but only got 0. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3014066176 bytes but only got 0. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3568893952 bytes but only got 23144. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-pac

Train Epoch: 3 [21760/25000 (87%)]	Loss: 0.207959
Train Epoch: 3 [23040/25000 (92%)]	Loss: 0.119673
Train Epoch: 3 [24320/25000 (97%)]	Loss: 0.120611
              precision    recall  f1-score   support

           7       1.00      1.00      1.00         2
           8       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         1
          25       0.00      0.00      0.00         1
          30       1.00      1.00      1.00         1
          38       0.00      0.00      0.00         1
          41       0.00      0.00      0.00         1
          48       1.00      1.00      1.00         2
          56       0.00      0.00      0.00         0
          70       1.00      1.00      1.00         1
          75       0.00      0.00      0.00         0
          76       1.00      1.00      1.00         1
          82       0.00      0.00      0.00         1
          84       1.00      1.00      1.00         1
          89       1.00      0.50      

In [ ]:

plt.figure(figsize=(5,3))
plt.plot(np.arange(1,num_epochs+1), lossv)
plt.title('validation loss')

plt.figure(figsize=(5,3))
plt.plot(np.arange(1,num_epochs+1), accv)
plt.title('validation accuracy');
     